# Laboratorio 11

La finalidad de este laboratorio es tener un mejor manejo de las herramientas que nos ofrece Scikit-Learn, como los _transformers_ y _pipelines_.  Usaremos el dataset [The Current Population Survey (CPS)](https://www.openml.org/d/534) que consiste en predecir el salario de una persona en función de atributos como la educación, experiencia o edad.

In [73]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml # trae la data de openml
from sklearn.model_selection import train_test_split

%matplotlib inline

Como siempre, un pequeño análisis descriptivo

In [74]:
survey = fetch_openml(data_id=534, as_frame=True)

In [75]:
X = survey.data[survey.feature_names]   #lo creo de la data
X.head()

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8.0,no,female,21.0,not_member,35.0,Hispanic,Other,Manufacturing,Married
1,9.0,no,female,42.0,not_member,57.0,White,Other,Manufacturing,Married
2,12.0,no,male,1.0,not_member,19.0,White,Other,Manufacturing,Unmarried
3,12.0,no,male,4.0,not_member,22.0,White,Other,Other,Unmarried
4,12.0,no,male,17.0,not_member,35.0,White,Other,Other,Married


In [76]:
X.describe(include="all").T.fillna("")

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
EDUCATION,534.0,,,,13.0187,2.61537,2,12,12,15,18
SOUTH,534.0,2,no,378,,,,,,,
SEX,534.0,2,male,289,,,,,,,
EXPERIENCE,534.0,,,,17.8221,12.3797,0,8,15,26,55
UNION,534.0,2,not_member,438,,,,,,,
AGE,534.0,,,,36.8333,11.7266,18,28,35,44,64
RACE,534.0,3,White,440,,,,,,,
OCCUPATION,534.0,6,Other,156,,,,,,,
SECTOR,534.0,3,Other,411,,,,,,,
MARR,534.0,2,Married,350,,,,,,,


In [77]:
y = survey.target
y.head()

0    5.10
1    4.95
2    6.67
3    4.00
4    7.50
Name: WAGE, dtype: float64

Y la posterior partición _train/test_.

In [78]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

## Ejercicio 1

(1 pto)

_One-Hot Encode_ es una técnica que a partir de una _feature_ categórica generar múltiples columnas, una por categoría.

* Define el transformador `ohe_sex` utilizando `OneHotEncoder` con atributos `drop="if_binary"` y `sparse=False`, luego ajusta y transforma el dataframe `X` solo con la columna `SEX`.
* Define el transformador `ohe_race` utilizando `OneHotEncoder` con atributos `drop="if_binary"` y `sparse=False`, luego ajusta y transforma el dataframe `X` solo con la columna `RACE`.

In [121]:
from sklearn.preprocessing import OneHotEncoder  

In [143]:
ohe_sex = OneHotEncoder(drop="if_binary",sparse=False)
ohe_sex=pd.DataFrame(ohe_sex.fit_transform(X[['SEX']]))
ohe_sex= X.join(ohe_sex).rename(columns={0:'Married(0.0)/Unmarried(1.0)'})

#No utilizo get_dummies ya que aplica drop="if_binary"
#dum_sex = pd.get_dummies(ohe_sex['SEX'], prefix="Sex_is" )
#ohe_sex = X.join(dum_sex)
ohe_sex

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,AGE,RACE,OCCUPATION,SECTOR,MARR,Married(0.0)/Unmarried(1.0)
0,8.0,no,female,21.0,not_member,35.0,Hispanic,Other,Manufacturing,Married,0.0
1,9.0,no,female,42.0,not_member,57.0,White,Other,Manufacturing,Married,0.0
2,12.0,no,male,1.0,not_member,19.0,White,Other,Manufacturing,Unmarried,1.0
3,12.0,no,male,4.0,not_member,22.0,White,Other,Other,Unmarried,1.0
4,12.0,no,male,17.0,not_member,35.0,White,Other,Other,Married,1.0
...,...,...,...,...,...,...,...,...,...,...,...
529,18.0,no,male,5.0,not_member,29.0,White,Professional,Other,Unmarried,1.0
530,12.0,no,female,33.0,not_member,51.0,Other,Professional,Other,Married,0.0
531,17.0,no,female,25.0,member,48.0,Other,Professional,Other,Married,0.0
532,12.0,yes,male,13.0,member,31.0,White,Professional,Other,Married,1.0


In [147]:
race=('Hispanic','White','Other')
ohe_race = OneHotEncoder(categories='auto',drop="if_binary",sparse=False) #drop="if_binary"  No aplica al haber 3 categorias
ohe_race=pd.DataFrame(ohe_race.fit_transform(X[['RACE']]) )
ohe_race=X.join(ohe_race)

#En esta transformacion uso get_durmies ya que no aplica drop="if_binary"
dum_race = pd.get_dummies(ohe_race['RACE'], prefix="Race_is" ) # Defino los nombres las nuevas filas por categoria
ohe_race = X.join(dum_race)
ohe_race

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,AGE,RACE,OCCUPATION,SECTOR,MARR,Race_is_Hispanic,Race_is_Other,Race_is_White
0,8.0,no,female,21.0,not_member,35.0,Hispanic,Other,Manufacturing,Married,1,0,0
1,9.0,no,female,42.0,not_member,57.0,White,Other,Manufacturing,Married,0,0,1
2,12.0,no,male,1.0,not_member,19.0,White,Other,Manufacturing,Unmarried,0,0,1
3,12.0,no,male,4.0,not_member,22.0,White,Other,Other,Unmarried,0,0,1
4,12.0,no,male,17.0,not_member,35.0,White,Other,Other,Married,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,18.0,no,male,5.0,not_member,29.0,White,Professional,Other,Unmarried,0,0,1
530,12.0,no,female,33.0,not_member,51.0,Other,Professional,Other,Married,0,1,0
531,17.0,no,female,25.0,member,48.0,Other,Professional,Other,Married,0,1,0
532,12.0,yes,male,13.0,member,31.0,White,Professional,Other,Married,0,0,1


__Pregunta:__ ¿Por qué las transformaciones resultantes tiene diferente cantidad de columnas?

__Respuesta:__ Por que en la primera tranfomacion se aplica a una feature binaria (2 categorias) por lo que al usar drop="if_binary" entrega una sola columna de 0 y 1, es decir es female o male. En cambio en la segunda transformacion hay 3 categorias por lo que drop="if_binary" no aplica y se generan 3 columnas, una por categoria donde si aparece un 1 pertenece a esta y si aparece un 0 no pertenece.

## Ejercicio 2

(1 pto)

Realizar _One-Hot-Encoding_ para cada una de las columnas categóricas y luego unirlas en un nuevo array o dataframe es tedioso, poco escablable y probablemente conlleve a errores. La función `make_column_transformer` permite automatizar este proceso en base a aplicar transformadores a distintas columnas.

* `categorical_columns` debe ser una lista con todos los nombres de columnas categóricas del dataframe `X`.
* `numerical_columns` debe ser una lista con todos los nombres de columnas numéricas del dataframe `X`.
* Define `preprocessor` utilizando `make_column_transformer` tal que:
    - A las columnas categóricas se les aplique `OneHotEncoder` con el argumento `drop="if_binary"`
    - El resto de las columnas se mantena igual. Hint: Revisar la documentación del argumento `remainder`.
* Finalmente define  `X_processed` al ajustar y transformar el dataframe `X` utilizando `preprocessor` 

In [154]:
from sklearn.compose import make_column_transformer

categorical_columns = ['SOUTH','SEX','UNION','RACE','OCCUPATION','SECTOR','MARR']
    
numerical_columns = ['EDUCATION','EXPERIENCE','AGE']

preprocessor = make_column_transformer(
   (OneHotEncoder(drop="if_binary"), categorical_columns),
    remainder='passthrough' #las columnas no especificadas se mantienen igual, en este caso las numerical_columns.
)

X_processed = pd.DataFrame(preprocessor.fit_transform(X))

X_processed=X.join(X_processed)

#No utilizo get_dummies ya que aplica a algunas columnas drop="if_binary"
#dum_south = pd.get_dummies(X_processed['SOUTH'], prefix="SOUTH")
#dum_sex = pd.get_dummies(X_processed['SEX'], prefix="SEX")
#dum_union = pd.get_dummies(X_processed['UNION'], prefix="UNI")
#dum_race = pd.get_dummies(X_processed['RACE'], prefix="RACE")
#dum_ocup = pd.get_dummies(X_processed['OCCUPATION'], prefix="OCP")
#dum_sector = pd.get_dummies(X_processed['SECTOR'], prefix="SECT")
#dum_marr = pd.get_dummies(X_processed['MARR'], prefix="MARR")
#X_processed = X.join(dum_south).join(dum_sex).join(dum_union).join(dum_race).join(dum_ocup).join(dum_sector).join(dum_marr)

X_processed

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,AGE,RACE,OCCUPATION,SECTOR,MARR,...,9,10,11,12,13,14,15,16,17,18
0,8.0,no,female,21.0,not_member,35.0,Hispanic,Other,Manufacturing,Married,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,21.0,35.0
1,9.0,no,female,42.0,not_member,57.0,White,Other,Manufacturing,Married,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,42.0,57.0
2,12.0,no,male,1.0,not_member,19.0,White,Other,Manufacturing,Unmarried,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,12.0,1.0,19.0
3,12.0,no,male,4.0,not_member,22.0,White,Other,Other,Unmarried,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,12.0,4.0,22.0
4,12.0,no,male,17.0,not_member,35.0,White,Other,Other,Married,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0,17.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,18.0,no,male,5.0,not_member,29.0,White,Professional,Other,Unmarried,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,18.0,5.0,29.0
530,12.0,no,female,33.0,not_member,51.0,Other,Professional,Other,Married,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0,33.0,51.0
531,17.0,no,female,25.0,member,48.0,Other,Professional,Other,Married,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,17.0,25.0,48.0
532,12.0,yes,male,13.0,member,31.0,White,Professional,Other,Married,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0,13.0,31.0


In [155]:
print(f"X_processed tiene {X_processed.shape[0]} filas y {X_processed.shape[1]} columnas.")  # revisa que cantidad de 

X_processed tiene 534 filas y 29 columnas.


__COMENTARIOS:__

Al contabilizar las nuevas columnas segun categorias y aplicando a las categorias binarias drop="if_binary" se contabilizan 16 nuevas columnas,
con remainder='passthrough' se mantienen las 3 columnas numericas y mas las 10 columnas originales dan 29 columnas en total para el nuevo DF.

## Ejercicio 3

(1 pto)

Sucede un fenómeno similar al aplicar transformaciones al vector de respuesta. En ocasiones es necesario transformarlo pero que las predicciones sean en la misma escala original. `TransformedTargetRegressor` juega un rol clave, pues los insumos necesarios son: un estimador, la función y la inversa para aplicar al vector de respuesta.

Define `ttr` como un `TransformedTargetRegressor` tal que:
* El regresor sea un modelo de regresión Ridge y parámetro de regularización `1e-10`.
* La función para transformar sea logaritmo base 10. Hint: `NumPy` es tu amigo.
* La función inversa sea aplicar `10**x`. Hint: Revisa el módulo `special` de `SciPy` en la sección de _Convenience functions_.

In [127]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import Ridge
from scipy import special
from scipy.special import exp10

In [128]:
ttr = TransformedTargetRegressor(regressor=Ridge(alpha=1e-10),
        func=np.log10, inverse_func= special.exp10
    )
ttr

TransformedTargetRegressor(func=<ufunc 'log10'>, inverse_func=<ufunc 'exp10'>,
                           regressor=Ridge(alpha=1e-10))

Ajusta el modelo con los datos de entrenamiento

In [129]:
ttr.fit(X_train, y_train)

ValueError: could not convert string to float: 'no'

Lamentablemente lanza un error :(

Prueba lo siguiente:

In [130]:
ttr.fit(X_train.select_dtypes(include="number"), y_train)

TransformedTargetRegressor(func=<ufunc 'log10'>, inverse_func=<ufunc 'exp10'>,
                           regressor=Ridge(alpha=1e-10))

In [131]:
ttr.score(X_train.select_dtypes(include="number"), y_train)

0.14236032206128302

In [132]:
ttr.regressor_.coef_

array([ 0.05893951,  0.02705054, -0.02174244])

__Pregunta:__ ¿Por qué falló el primer ajusto? ¿Qué tiene de diferente el segundo?

__Respuesta:__ Falló por que no puede convertir el tipo de dato string a flotante. En cambio en el segundo al usar select_dtypes(include="number") selecciona el tipo de dato numerico (float) y deja afuera los string.

## Ejercicio 4

(1 pto)

Ahora agreguemos todos los ingredientes a la juguera.

* Define `model` utilizando `make_pipeline` con los insumos `preprocessor` y `ttr`.
* Ajusta `model` con los datos de entrenamiento.
* Calcula el error absoluto medio con los datos de test.

In [135]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import median_absolute_error

model = make_pipeline(
   preprocessor,ttr
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = median_absolute_error(y_test, y_pred)

print(f"El error absoluto medio obtenido es {mae}")

El error absoluto medio obtenido es 2.29854679219015
